In [62]:
import cv2
import numpy as np
from enum import Enum
import colorsys
import moviepy.editor as mpe
import moviepy
from typing import *
import random
import pandas as pd
import json
import torch

ModuleNotFoundError: No module named 'torch'

In [2]:
import IPython
def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [17]:
exp_1_1_data = pd.read_csv('./resources/experiment_1_1_1.csv')
orange_rgbs = exp_1_1_data.loc[lambda r:r['ans'] == 0, :]

ans                                                           0
circle_rgb    [0.9795646896652704, 0.42022575630354925, 0.07...
bg_rgb        [0.6902299453895665, 0.6902299453895665, 0.690...
Name: 3, dtype: object

In [60]:
NUM_POSITIONS = 16  
IMG_SIZE = 224
ROW_LEN = NUM_POSITIONS**0.5
RADIUS = int(IMG_SIZE / (2 * ROW_LEN)) # 28
assert ROW_LEN == 4
assert RADIUS == 28


class ColorOption:
    def __init__(self, circle_rgb, bg_rgb):
        self.circle_rgb = circle_rgb
        self.bg_rgb = bg_rgb
    
    
def generate_sample():
    pos = random.randrange(0, NUM_POSITIONS)
    color = random_color()
    circle_img = circle_img(pos, color.circle_rgb)
  

def random_orange() -> ColorOption:
    # TODO replace with function.
    orange_rgbs = exp_1_1_data.loc[lambda r:r['ans'] == 0, :]
    idx = random.randrange(0, orange_rgbs.shape[0])
    circle_rgb  = json.loads(orange_rgbs.iloc[idx]['circle_rgb'])
    bg_rgb = json.loads(orange_rgbs.iloc[idx]['bg_rgb'])
    circle_rgb = np.array(circle_rgb)*255
    bg_rgb = np.array(bg_rgb)*255
    return ColorOption(circle_rgb, bg_rgb)

    
def random_brown() -> ColorOption:
    # TODO replace with function.
    brown_rgbs = exp_1_1_data.loc[lambda r:r['ans'] == 1, :]
    idx = random.randrange(0, brown_rgbs.shape[0])
    circle_rgb  = json.loads(brown_rgbs.iloc[idx]['circle_rgb'])
    bg_rgb = json.loads(brown_rgbs.iloc[idx]['bg_rgb'])
    circle_rgb = np.array(circle_rgb)*255
    bg_rgb = np.array(bg_rgb)*255
    return ColorOption(circle_rgb, bg_rgb)


def random_color() -> ColorOption:
    fctns = [random_orange, random_brown]
    idx = random.randrange(0, 2)
    label = "orange" if idx == 0 else "brown"
    f = fctns[idx]
    return f(), label


def coords(position_idx : int) -> Tuple[int, int]:
    """Calculates the (y, x) coordinates of a position index.
    
    Origin is top left.
    """
    if not 0 <= position_idx < NUM_POSITIONS:
        raise Exception(f'Position index must be within 0 and {NUM_POSITIONS}. '
                        'Got: {position_idx}')
    row = position_idx // ROW_LEN
    col = position_idx % ROW_LEN
    y = int(RADIUS * (row * 2 + 1))
    x = int(RADIUS * (col * 2 + 1))
    return (y, x)
     
    
def to_cv2_coords(numpy_coords):
    return (numpy_coords[1], numpy_coords[0])
    
    
def circle_img(position_idx, circle_color, bg_color):
    img = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
    img[:] = bg_color
    center = to_cv2_coords(coords(position_idx))
    img = cv2.circle(img, center, RADIUS, circle_color, thickness=-1, lineType=cv2.LINE_AA)
    return img


def create_samples(num_samples):
    frames = []
    labels = []
    for n in range(num_samples):
        color, label = random_color()
        pos = random.randrange(0, NUM_POSITIONS)
        frame = circle_img(pos, color.circle_rgb, color.bg_rgb)
        frames.append(frame)
        labels.append(label)
    return frames, labels
    

In [61]:
def demo_data():
    FPS = 2
    #frames = [circle_img(p, (255, 255, 255), (50, 20, 20)) for p in range(NUM_POSITIONS)]
    #labels = ['WB-0']*len(frames)
    frames, labels = create_samples(30)
    x_clip = mpe.ImageSequenceClip(frames, fps=2)
    y_clip = mpe.TextClip('WB-0', font='DejaVu-Sans')

    class FrameText(mpe.VideoClip):
        def __init__(self, text, fps):
            def make_frame(f):
               return mpe.TextClip(text[int(f)], font='DejaVu-Sans', color='white').get_frame(f)
            self.duration = 1.0 * len(text) / fps
            mpe.VideoClip.__init__(self, make_frame=make_frame, duration=self.duration)

    y_clip =   FrameText(labels, FPS)
    label_clip = mpe.CompositeVideoClip([mpe.ImageClip(np.zeros((IMG_SIZE, IMG_SIZE, 3)), duration=5), y_clip])
    comp_clip = mpe.clips_array([[y_clip],[x_clip]])
    return comp_clip
clip = demo_data() 
clip.ipython_display()

t:  20%|██        | 6/30 [00:00<00:00, 59.13it/s, now=None]

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [ ]:
class 